# Grid class

## Atrributes

grid.**nc**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    $N_c$: Number of grid points per dimension.
    
    
grid.**boxsize**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    $L_{box}$: Length of the cubic box on a side
    
grid.**x0**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    Coordinate of the corner of the box

grid.**offset**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    Position of the grid points in units of the grid spacing $(0 \le \mathrm{offset} < 1)$.
    <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    The coordinate of the grid points are $x0 + (i + \mathrm{offset})\Delta x$; $\Delta x = \textrm{boxsize}/\textrm{nc}$ for integers $0 \le i < \mathrm{nc}$.
    
grid.**mode**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    'real-space' or 'fourier-space'

## Create a Grid object

In [1]:
import lssps
nc = 2 # number of grids per dimension
boxsize = 100.0

# zeros() create a grid set to zero
grid = lssps.grid.zeros(nc, boxsize, x0=(0, 0, 0), interlacing=True)

# This is equivalent to creating an uninitialised grid and clear it
grid = lssps.grid.empty(nc, boxsize, x0=(0, 0, 0), interlacing=True)
grid.clear()

# A grid with same parameters shifted by 0.5 grid spacing is created
# if interlacing=True.
print(grid)
print(grid.shifted) # shifted grid
print('grid.x0 =', grid.offset)
print('grid.shfted.x0 =', grid.shifted.offset)


grid.x0 = 0.0
grid.shfted.x0 = -0.5


## Field values at grid points

In [2]:
# grid[:] is gives an np.array of discrete field defined grid points
print('grid[0, 0, 0] =', grid[0, 0, 0])
print('grid[:].shape =', grid[:].shape)

# the value can be assigned
grid[:] = np.ones((2, 2, 2))
print('grid[0, 0, 0] =>', grid[0, 0, 0])

grid[0, 0, 0] = 0.0
grid[:].shape = (2, 2, 2)
grid[0, 0, 0] => 1.0


## Fast Fourier Transform (FFT)

The grid can be Fourier transformd to Fourier transform and inverse Fourier transformed back to real space. The grid shape is (nc, nc, nc) in real space, and (nc, nc, nc//2 + 1) in Fourier space.

In [20]:
nc = 4
grid = lssps.grid.zeros(nc, boxsize, x0=(0, 0, 0), interlacing=True)
grid[:] = np.ones((nc, nc, nc))
print(grid[0,0,:4])
print(grid.mode)
print(grid[:].dtype, grid[:].shape)

grid.fft()

print(grid.mode)
print(grid[:].dtype, grid[:].shape)

grid.fft_inverse()
print(grid[0,0,:4])
# Fourier transform and inverse Transform gives back nc**3 times the original

[1. 1. 1. 1.]
real-space
float64 (4, 4, 4)
fourier-space
complex128 (4, 4, 3)
[64. 64. 64. 64.]


## Mass assignment

grid.**assign_density**(_xyz_, *, _weight_=None, _nbar_=None, _mas_='CIC')<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Assign number density to the grid.<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
_xyz_ (array): Cartesian coordinate of the object. Must have 3 columns.<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
_weight_ (array): weights of the object $w_i$. Default weight is 1 if this is set to None.
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
_nbar_ (array): mean densities at object position $\bar{n}_i$. Default value is 1 if this is set to None.
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
_mas_ (str): mass assignment scheme 'NGP', 'CIC', or 'TSC'.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
The unit of the density is the number of weighted objectes per cell, $\sum w_i$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
This function does not clear the grid to zero before assigning the density.

The mass assignment also updates the following statistics:

grid.**w_sum**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$ \sum_i w_i $

grid.**w2_sum**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$ \sum_i w_i^2 $

grid.**nw2_sum**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$ \sum_i n_i w_i^2 $

These sums are added to the values before mass assignment.

grid.**clear()**<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
reset the grid and its statistics to 0 and mode to 'real-space'

## Fluctuation

grid.**compute_fluctuation**(*grid_rand* = None)<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
subtracts the random density field from the density grid $n$,
$$ n(\vec{x}) \leftarrow F(\vec{x}) =  n(\vec{x}) - \alpha n_{rand}(\vec{x}) $$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
where,
$\alpha = \sum_i w_i \big/ \sum_i w_{rand,i} = \textrm{grid.w_sum} / \textrm{grid_rand.w_sum}$

This function also computes:

grid.**pk_normalisation** = $1/\mathcal{N}$
$$ \mathcal{N} = \sum_{i} \bar{n}_{rand, i} w_{rand, i}^2$$ 

grid.**shot_noise** = $\mathcal{S}$
$$ \mathcal{S} = (1 + \alpha) \mathcal{N}^{-1} \sum_{rand, i} w_{rand, i}^2$$

These values are used when the power spectrum:

$$ \tilde{P}(\vec{k}) = \mathcal{N}^{-1} \langle F(k) F(k)^* \rangle - \mathcal{S}$$

### Fluctuation for homogenious data

`grid_rand` can be None when the data is statistically homogeneous (simulation data in a periodic box). Power spectrum in a homogeneous box is defined as,

$$ P(k) = \frac{1}{V} \langle \delta(k) \delta(k)^* \rangle $$

where, $V = \textrm{boxsize}^3 $ is the volume of the box and
Fourier transform,
$$ \delta(\vec{k}) = \int_V \! d^3 x \, \delta(\vec{x})  e^{-i\vec{k}\cdot\vec{x}} $$

This corresponds to
$$ F(\vec{x}) = n(\vec{x}) - \bar{n}, $$
$$ \quad \bar{n} = \sum_i w_i / N_c^3, $$
$$ \mathcal{N} = \left( \sum_i w_i \right)^2 \big/ V $$
and the shot noise is
$$ \mathcal{S} = V \frac{\sum_i w_i^2}{(\sum_i w)^2} = \mathcal{N}^{-1} \sum_i w_i^2$$